<a href="https://colab.research.google.com/github/DenZhid/DeepLearningLabs/blob/master/lab3/Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Загрузка библиотеки и необходимых зависимостей**

In [1]:
!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install stable-baselines3[extra] ale-py==0.7.4
!pip install stable-baselines3[extra] box2d box2d-kengz
!apt install swig cmake

Reading package lists... Done
Building dependency tree       
Reading state information... Done
freeglut3-dev is already the newest version (2.8.1-3).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
xvfb is already the newest version (2:1.19.6-1ubuntu4.10).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
swig is already the newest version (3.0.12-1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [45]:
import base64
import gym
import os
import numpy as np
import matplotlib.pyplot as plt

from IPython import display as ipythondisplay
from google.colab import files
from pathlib import Path

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common import results_plotter
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

# Нужно для записи видео
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [39]:
# Функции записи видео
# При изменении окружения, в котором обучается сеть также необходимо поменять её 
# и здесь
def record_video(env_id, model, video_length=500, prefix='', video_folder='videos/'):
  eval_env = DummyVecEnv([lambda: gym.make('SeaquestNoFrameskip-v4')])
  eval_env = VecVideoRecorder(eval_env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = eval_env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs)
    obs, _, _, _ = eval_env.step(action)

  eval_env.close()

def show_videos(video_path='', prefix=''):
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

**Простейшая имплементация нейронной сети DQN**

In [5]:
# Для данной нейронной сети требуется слишком много памяти. Предоставляемых Colab
# ресурсов не хватает
# env = make_atari_env('PongNoFrameskip-v4', n_envs=1, seed=0)
# env = VecFrameStack(env, n_stack=1)
# model = DQN('CnnPolicy', env, verbose=1)
# model.learn(total_timesteps=10000)

**Имплементация другого алгоритма A2C**

In [35]:
# Чтобы изменить окружение, введите нужно окружение в переменную ниже
env = gym.make('SeaquestNoFrameskip-v4')

model = A2C('CnnPolicy', env, verbose=1)
model.learn(total_timesteps=10000)

Using cpu device
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
-------------------------------------
| time/                 |           |
|    fps                | 20        |
|    iterations         | 100       |
|    time_elapsed       | 24        |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -2.89     |
|    explained_variance | -8.87e-05 |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | 45.4      |
|    value_loss         | 311       |
-------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 20       |
|    iterations         | 200      |
|    time_elapsed       | 48       |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -2.89    |
|    explained_variance | -10.6    |
|    learning_rate      | 0.0007   |
|    

In [ ]:
# Выберите модели, которые нужно загрузить
files.upload()

In [ ]:
!du -h a2c*

In [ ]:
trained_model = A2C.load("a2c_pong", verbose=1)
env = gym.make('SeaquestNoFrameskip-v4')
trained_model.set_env(env)

In [47]:
# Изменяйте id окружения, чтобы создать нужные видео. Также смотри в раздел
# описания методов для записи и просмотра видео
record_video('SeaquestNoFrameskip-v4', model, video_length=1000, prefix='a2c-SeaquestNoFrameskip-v4')

Saving video to /content/videos/a2c-SeaquestNoFrameskip-v4-step-0-to-step-1000.mp4


In [48]:
# Измените префикс, чтобы посмотреть видео другой архитекутры сети
show_videos('videos', prefix='a2c')

In [ ]:
# Данная оценка может занять очень много времени
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")